In [1]:
#autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [3]:
import warcraftlogs
from warcraftlogs.constants import TOKEN_DIR

from warcraftlogs import WarcraftLogsClient

client = WarcraftLogsClient(token_dir=TOKEN_DIR)

In [63]:
warcraftlogs_url = "https://www.warcraftlogs.com/reports/vWLfFkZ71V3tQAnb?fight=39&type=damage-done&source=12"

# extract report code, fight id, and source id from url
# report code just the last part of the url
REPORT_CODE = re.search(r'/reports/(\w+)', warcraftlogs_url).group(1)
# fight id is the last part of the url
FIGHT_ID = re.search(r'fight=(\d+)', warcraftlogs_url).group(1)
# source id is the last part of the url
SOURCE_ID = re.search(r'source=(\d+)', warcraftlogs_url).group(1)

# convert to int
SOURCE_ID = int(SOURCE_ID)
FIGHT_ID = int(FIGHT_ID)


In [27]:
from warcraftlogs.query.tables import get_table_data

damage_done_table_data_query = get_table_data(REPORT_CODE, FIGHT_ID, SOURCE_ID,
                                  data_type="DamageDone")
damage_done_table_data_query

buff_table_data_query = get_table_data(REPORT_CODE, FIGHT_ID, SOURCE_ID,
                                  data_type="Buffs")
buff_table_data_query

'\n    query {\n        reportData {\n            report(code: "vWLfFkZ71V3tQAnb") {\n                table(\n                fightIDs: 39, dataType: Buffs, sourceID: 12\n                )\n            }\n        }\n    }\n    '

In [28]:
print(damage_done_table_data_query)


    query {
        reportData {
            report(code: "vWLfFkZ71V3tQAnb") {
                table(
                fightIDs: 39, dataType: DamageDone, sourceID: 12
                )
            }
        }
    }
    


In [29]:
damage_done_table_data_resp = client.query_public_api(damage_done_table_data_query)


In [33]:
buff_table_data_resp = client.query_public_api(buff_table_data_query) 

In [39]:
#10162683 millisecond in minutes
buff_table_data_resp['data']['reportData']['report']['table']['data']['totalTime'] / 1000 / 60

6.034516666666667

In [37]:
buff_table_data_resp['data']['reportData']['report']['table']['data']

{'auras': [{'name': 'Elemental Resistance',
   'guid': 462568,
   'type': 1,
   'abilityIcon': 'spell_fireresistancetotem_01.jpg',
   'totalUptime': 150110,
   'totalUses': 65,
   'bands': [{'startTime': 9363261, 'endTime': 9366270},
    {'startTime': 9372074, 'endTime': 9375460},
    {'startTime': 9379482, 'endTime': 9382507},
    {'startTime': 9389984, 'endTime': 9392966},
    {'startTime': 9421305, 'endTime': 9424305},
    {'startTime': 9426661, 'endTime': 9432140},
    {'startTime': 9437924, 'endTime': 9440897},
    {'startTime': 9449221, 'endTime': 9452334},
    {'startTime': 9460309, 'endTime': 9464374},
    {'startTime': 9471314, 'endTime': 9474290},
    {'startTime': 9479238, 'endTime': 9482251},
    {'startTime': 9482666, 'endTime': 9486949},
    {'startTime': 9491280, 'endTime': 9494267},
    {'startTime': 9502334, 'endTime': 9510288},
    {'startTime': 9510899, 'endTime': 9516607},
    {'startTime': 9517776, 'endTime': 9520799},
    {'startTime': 9549753, 'endTime': 9557351}

In [69]:
from warcraftlogs.client import WarcraftLogsClient
from warcraftlogs.query.player_analysis import *

In [84]:
player = get_player_details(client, report_code=REPORT_CODE, fight_id=FIGHT_ID, source_id=SOURCE_ID)

In [85]:
player 

PlayerDetails(name='Shunwalker', id=12, spec_name='Retribution', role='dps', class_name='Paladin', item_level=658, bracket=9)

In [205]:
fight = get_fight_info(client, report_code=REPORT_CODE, fight_id=FIGHT_ID)


    {
        reportData {
            report(code: "vWLfFkZ71V3tQAnb") {
                fights(fightIDs: [39]) {
                    encounterID
                    name
                    difficulty
                    averageItemLevel
                    startTime
                    endTime
                    gameZone {
                        id
                        name
                    }
                }
                zone {
                    id
                    name
                }
            }
        }
    }
    


In [217]:
fight['fights'][0]

{'encounterID': 3015,
 'name': "Mug'Zee, Heads of Security",
 'difficulty': 4,
 'averageItemLevel': 661.2222290039062,
 'startTime': 9347971,
 'endTime': 9710042,
 'gameZone': {'id': 2769, 'name': 'Liberation of Undermine'}}

In [80]:
similar = get_similar_players(
        client,
        encounter_id=fight['encounterID'],
        spec_id=player.spec_name,
        bracket=player.bracket,
        difficulty=fight['difficulty']
    )

In [225]:
def generate_ranking_query(player: PlayerDetails, fight: dict):
    query = """
    query GetDungeonRankings($zoneID: Int!) {{
      worldData {{
        zone(id: $zoneID) {{
          name
          encounters(id: [3015]) {{
            id
            name
            characterRankings(
              className: "{}"
              specName: "{}"
              bracket: {}
              includeCombatantInfo: true,
              leaderboard: LogsOnly
            )
          }}
        }}
      }}
    }}
    """.format(player.class_name, player.spec_name, player.bracket)
    
    variables = {
        "zoneID": fight['zone']['id'],
        #"encounterID": fight['fights'][0]['encounterID']
    }
    return query, variables

def generate_ranking_query(**kwargs):
    source_filter_str = ""
    for key, value in kwargs.items():
        source_filter_str += f"{key}: {value},\n"
    source_filter_str = source_filter_str[:-2]  # remove last comma and newline
    query = f"""
    query GetDungeonRankings($encounterID: Int!) {{
      worldData {{
          encounter(id: $encounterID) {{
            id
            name
            characterRankings(
              includeCombatantInfo: false,
              leaderboard: LogsOnly,
              {source_filter_str}
            )
          }}
        }}
      }}
    """
    return query

def generate_ranking_query_from_player_and_fight(player: PlayerDetails, fight: dict):
    filters = {
        "bracket": player.bracket,
        "className": f'"{player.class_name}"',
        "specName": f'"{player.spec_name}"',
        "difficulty": fight['difficulty']
    }
    query_generated = generate_ranking_query(**filters)
    variables = {
        "encounterID": fight['encounterID']
    }
    return query_generated, variables

In [226]:
query, variables = generate_ranking_query_from_player_and_fight(player, fight['fights'][0])
print(query)
ranking_resp = client.query_public_api(query, variables)
ranking_resp


    query GetDungeonRankings($encounterID: Int!) {
      worldData {
          encounter(id: $encounterID) {
            id
            name
            characterRankings(
              includeCombatantInfo: false,
              leaderboard: LogsOnly,
              bracket: 9,
className: "Paladin",
specName: "Retribution",
difficulty: 4
            )
          }
        }
      }
    


{'data': {'worldData': {'encounter': {'id': 3015,
    'name': 'Mug’Zee, Heads of Security',
    'characterRankings': {'page': 1,
     'hasMorePages': True,
     'count': 100,
     'rankings': [{'name': '神拳永远滴神',
       'class': 'Paladin',
       'spec': 'Retribution',
       'amount': 2316743.7773016,
       'hardModeLevel': 0,
       'duration': 388256,
       'startTime': 1743041900353,
       'report': {'code': 'Xbxhcrw7gmYfWnJ8',
        'fightID': 31,
        'startTime': 1743037211626},
       'guild': {'id': 526549, 'name': '如家', 'faction': 1},
       'server': {'id': 676, 'name': '影之哀伤', 'region': 'CN'},
       'bracketData': 659,
       'faction': 1,
       'size': 30},
      {'name': 'Garbageguy',
       'class': 'Paladin',
       'spec': 'Retribution',
       'amount': 2288271.1212254,
       'hardModeLevel': 0,
       'duration': 328248,
       'startTime': 1743006010255,
       'report': {'code': 'PVwfmJgCYB86yDjt',
        'fightID': 52,
        'startTime': 1743001179642

In [232]:
similar_player_rankings =ranking_resp['data']['worldData']['encounter']['characterRankings']['rankings']

In [248]:
similar_player_ranking = similar_player_rankings[0]
similar_player_ranking

{'name': '神拳永远滴神',
 'class': 'Paladin',
 'spec': 'Retribution',
 'amount': 2316743.7773016,
 'hardModeLevel': 0,
 'duration': 388256,
 'startTime': 1743041900353,
 'report': {'code': 'Xbxhcrw7gmYfWnJ8',
  'fightID': 31,
  'startTime': 1743037211626},
 'guild': {'id': 526549, 'name': '如家', 'faction': 1},
 'server': {'id': 676, 'name': '影之哀伤', 'region': 'CN'},
 'bracketData': 659,
 'faction': 1,
 'size': 30}

In [ ]:
similar_player_ranking_player_details =get_player_details(client, report_code=similar_player_ranking['report']['code'],
                    fight_id=similar_player_ranking['report']['fightID'])

In [256]:
# find the id of the player in the similar_player_ranking_player_details
def find_player_id_from_name(fight_player_details: list,
                             player_name: str):
    """
    fight_player_details is a list of dictionaries, each dictionary is a role and the player details
    """
    for role, player_details in fight_player_details.items():
        for player_detail in player_details:
            if player_detail['name'] == player_name:
                return player_detail['id']
    return None

similar_player_ranking_player_id = find_player_id_from_name(similar_player_ranking_player_details['data']['playerDetails'], similar_player_ranking['name'])
similar_player_ranking_player_id

1

In [276]:
from tqdm import tqdm
similar_player_report_info = []
for ranking in tqdm(similar_player_rankings[:10]):
    report_code = ranking['report']['code']
    fight_id = ranking['report']['fightID']
    ranking_player_details = get_player_details(client, report_code=report_code, fight_id=fight_id)
    player_source_id = find_player_id_from_name(ranking_player_details['data']['playerDetails'], ranking['name'])
    similar_player_report_info.append({
        'report_code': ranking['report']['code'],
        'fight_id': ranking['report']['fightID'],
        'player_source_id': player_source_id
    })



100%|██████████| 10/10 [00:02<00:00,  4.49it/s]


In [299]:
top_player_buff_info_lst = []
top_player_dmg_info_lst = []


for report_info in tqdm(similar_player_report_info[:10]):
    buff_table_data_query = get_table_data(report_info['report_code'], report_info['fight_id'], report_info['player_source_id'],
                                          data_type="Buffs", viewOptions=16)
    buff_table_data_resp = client.query_public_api(buff_table_data_query)
    top_player_buff_info_lst.append(buff_table_data_resp['data']['reportData']['report']['table']['data'])

    damage_done_table_data_query = get_table_data(report_info['report_code'], report_info['fight_id'], report_info['player_source_id'],
                                          data_type="DamageDone")
    damage_done_table_data_resp = client.query_public_api(damage_done_table_data_query)
    top_player_dmg_info_lst.append(damage_done_table_data_resp['data']['reportData']['report']['table']['data'])




100%|██████████| 10/10 [00:14<00:00,  1.48s/it]


In [388]:
def get_buff_info_df(buff_info: list):
    total_time = buff_info['totalTime']    
    buff_info_df = pd.DataFrame(buff_info['auras'])
    buff_info_df['total_time'] = total_time
    buff_info_df['up_time_pct'] = buff_info_df['totalUptime'] / total_time
    # round up_time_pct to 2 decimal places
    buff_info_df['up_time_pct'] = buff_info_df['up_time_pct'].round(2)
    # put ['name', 'up_time_pct'] in the first two columns and keep the rest of the columns in the same order
    buff_info_df = buff_info_df[['name', 'up_time_pct'] + [col for col in buff_info_df.columns if col not in ['name', 'up_time_pct']]]
    return buff_info_df


def get_damage_info_df(damage_data: dict):
    damage_total_time = damage_data['totalTime']
    damage_info_df = pd.DataFrame(damage_data['entries'])
    damage_info_df['dps'] = (damage_info_df['total']/damage_total_time*1000).round(0)
    damage_info_df['crit_pct'] = (damage_info_df['critHitCount']/damage_info_df['hitCount']).round(2)*100

    damage_info_df['hit_per_minute'] = (damage_info_df['hitCount']/damage_total_time*1000*60).round(2)
    return damage_info_df

In [404]:
top_player_buff_info_lst_clean =list(map(get_buff_info_df, top_player_buff_info_lst))
top_player_damage_info_lst_clean = list(map(get_damage_info_df, top_player_dmg_info_lst))

In [398]:
player_buff_info_df = get_buff_info_df(buff_table_data_resp['data']['reportData']['report']['table']['data'])
player_damage_info_df = get_damage_info_df(damage_done_table_data_resp['data']['reportData']['report']['table']['data'])

In [412]:
player_damage_info_df[['name','dps', 'hitCount', 'critHitCount', 'crit_pct', 'uses','hit_per_minute',]].\
    sort_values("dps", ascending=False)

,name,dps,hitCount,critHitCount,crit_pct,uses,hit_per_minute
14,Empyrean Hammer,457979.0,830,265,32.0,134.0,136.37
10,Final Verdict,338158.0,121,35,29.0,101.0,19.88
8,Hammer of Light,317410.0,32,17,53.0,27.0,5.26
11,Execution Sentence,182515.0,9,0,0.0,12.0,1.48
1,Blade of Justice,133849.0,51,15,29.0,51.0,8.38
4,Wake of Ashes,124442.0,36,11,31.0,12.0,5.91
12,Divine Storm,118720.0,64,21,33.0,23.0,10.51
2,Judgment,105988.0,82,27,33.0,57.0,13.47
0,Divine Hammer,89649.0,168,60,36.0,136.0,27.60
9,Crusading Strikes,86305.0,209,77,37.0,211.0,34.34


In [420]:
def compare_damage_info(base_df, compare_df):
    """
    Compare damage information between two players/specs
    
    Args:
        base_df (DataFrame): Base player's damage data
        compare_df (DataFrame): Comparison player's damage data
        
    Returns:
        DataFrame: Comparison metrics with absolute and relative differences
    """
    import numpy as np
    
    # Get common abilities between both players
    common_abilities = set(base_df['name']) & set(compare_df['name'])
    
    # Filter both dataframes to only include common abilities
    base_df = base_df[base_df['name'].isin(common_abilities)].sort_values('name').reset_index(drop=True)
    compare_df = compare_df[compare_df['name'].isin(common_abilities)].sort_values('name').reset_index(drop=True)
    
    # Create comparison dataframe
    comparison = pd.DataFrame()
    comparison['ability'] = base_df['name']
    
    # Calculate absolute differences
    metrics = ['dps', 'hit_per_minute'] #, 'critHitCount', 'crit_pct', 'uses',  'hitCount'
    for metric in metrics:
        comparison[f'{metric}_base'] = base_df[metric]
        comparison[f'{metric}_compare'] = compare_df[metric]
        comparison[f'{metric}_diff'] = compare_df[metric] - base_df[metric]
        
        # Calculate percentage difference
        comparison[f'{metric}_diff_pct'] = np.where(
            base_df[metric].notna() & (base_df[metric] != 0),
            ((compare_df[metric] - base_df[metric]) / base_df[metric] * 100),
            np.nan
        )
    
    # Format percentage columns
    pct_columns = [col for col in comparison.columns if 'pct' in col]
    for col in pct_columns:
        comparison[col] = comparison[col].round(1)
        
    # Format numeric columns
    numeric_columns = [col for col in comparison.columns if col not in ['ability'] + pct_columns]
    for col in numeric_columns:
        comparison[col] = comparison[col].round(1)
    
    return comparison

In [425]:
compare_damage_info(player_damage_info_df, top_player_damage_info_lst_clean[0]).\
    sort_values('dps_diff', ascending=False).\
    style.background_gradient(cmap='RdYlGn', subset=['dps_diff','hit_per_minute_diff'])

,ability,dps_base,dps_compare,dps_diff,dps_diff_pct,hit_per_minute_base,hit_per_minute_compare,hit_per_minute_diff,hit_per_minute_diff_pct
8,Final Verdict,338158.000000,519601.000000,181443.000000,53.700000,19.900000,31.800000,12.000000,60.100000
6,Empyrean Hammer,457979.000000,534227.000000,76248.000000,16.600000,136.400000,170.000000,33.600000,24.700000
4,Divine Hammer,89649.000000,115811.000000,26162.000000,29.200000,27.600000,34.200000,6.500000,23.700000
14,Wake of Ashes,124442.000000,148864.000000,24422.000000,19.600000,5.900000,7.300000,1.300000,22.800000
7,Execution Sentence,182515.000000,196759.000000,14244.000000,7.800000,1.500000,1.800000,0.400000,25.000000
3,Crusading Strikes,86305.000000,94296.000000,7991.000000,9.300000,34.300000,37.200000,2.900000,8.400000
2,Consecration,31401.000000,36646.000000,5245.000000,16.700000,80.700000,87.000000,6.300000,7.800000
0,Authority of Radiant Power,6945.000000,10444.000000,3499.000000,50.400000,1.000000,1.700000,0.700000,71.700000
1,Blade of Justice,133849.000000,133967.000000,118.000000,0.100000,8.400000,7.600000,-0.800000,-9.700000
13,Melee,0.000000,0.000000,0.000000,nan,0.000000,0.000000,0.000000,nan


In [332]:
def compare_buff_uptime(buff_info_df1: pd.DataFrame, buff_info_df2: pd.DataFrame):
    buff_info_df1 = buff_info_df1[['name', 'up_time_pct', 'totalUses', 'type', 'guid']]
    buff_info_df2 = buff_info_df2[['name', 'up_time_pct', 'totalUses', 'type', 'guid']]
    merged_df = buff_info_df1.merge(buff_info_df2, on=['name', 'guid'], suffixes=('_1', '_2'), how=
                               'inner')
    merged_df['up_time_pct_diff'] = merged_df['up_time_pct_1'] - merged_df['up_time_pct_2']
    merged_df['total_uses_diff'] = merged_df['totalUses_1'] - merged_df['totalUses_2']
    merged_df['abs_up_time_pct_diff'] = merged_df['up_time_pct_diff'].abs()
    return merged_df

# color the up_time_pct_diff column by the magnitude of the difference
compare_buff_uptime(player_buff_info_df, top_player_buff_info_lst_clean[1]).\
    sort_values(by='abs_up_time_pct_diff', ascending=False).head(10).\
    style.background_gradient(cmap='RdYlGn', subset=['abs_up_time_pct_diff'])

,name,up_time_pct_1,totalUses_1,type_1,guid,up_time_pct_2,totalUses_2,type_2,up_time_pct_diff,total_uses_diff,abs_up_time_pct_diff
4,Divine Hammer,0.700000,6,2,198034,0.900000,5,2,-0.200000,1,0.200000
7,Final Verdict,0.140000,16,2,383329,0.270000,22,2,-0.130000,-6,0.130000
13,Ascendance,0.220000,10,4,458524,0.320000,13,4,-0.100000,-3,0.100000
17,Ascendance,0.280000,13,4,458503,0.190000,8,4,0.090000,5,0.090000
15,Art of War,0.210000,53,2,406086,0.130000,28,2,0.080000,25,0.080000
19,Winning Streak!,0.890000,61,1,1216828,0.820000,57,1,0.070000,4,0.070000
35,Rush of Light,0.680000,35,1,407065,0.750000,45,1,-0.070000,-10,0.070000
10,Ascension,0.280000,13,4,458502,0.220000,10,4,0.060000,3,0.060000
31,Avenging Wrath,0.020000,1,2,454351,0.080000,3,2,-0.060000,-2,0.060000
18,Blessing of Dawn,0.360000,98,1,385127,0.300000,79,1,0.060000,19,0.060000


In [309]:
top_player_buff_info_lst_clean[1][['name', 'up_time_pct', 'totalUses']]

,name,up_time_pct,totalUses
0,Avenging Wrath,0.08,3
1,Flask of Alchemical Chaos,1.00,1
2,All in!,0.12,10
3,Divine Hammer,0.90,5
4,Shake the Heavens,0.98,25
5,Blessing of Dusk,0.97,74
6,Crusade,0.62,69
7,Empyreal Ward,0.02,1
8,Light's Deliverance,0.96,761
9,Divine Arbiter,0.96,613


In [307]:
top_player_buff_info_lst_clean[0]

,name,up_time_pct,guid,type,abilityIcon,totalUptime,totalUses,bands,total_time
0,Undisputed Ruling,0.58,432629,1,spell_holy_righteousfury.jpg,223794,32,"[{'startTime': 4695977, 'endTime': 4703974}, {...",388256
1,Art of War,0.19,406086,2,ability_paladin_artofwar.jpg,74719,49,"[{'startTime': 4706631, 'endTime': 4710122}, {...",388256
2,Loyal Customer,1.00,1225298,1,inv_inscription_tarot_earthquakecard.jpg,388255,1,"[{'startTime': 4688728, 'endTime': 5076983}]",388256
3,Crystallization,1.00,453250,1,inv_10_enchanting_crystal_color5.jpg,388255,1,"[{'startTime': 4688728, 'endTime': 5076983}]",388256
4,Divine Shield,0.02,642,2,spell_holy_divineshield.jpg,8024,1,"[{'startTime': 4884568, 'endTime': 4892592}]",388256
5,Sacrosanct Crusade,0.27,461867,2,inv_plate_raidpaladinprimalist_d_01_cape.jpg,104646,41,"[{'startTime': 4693629, 'endTime': 4705974}, {...",388256
6,Final Verdict,0.35,383329,2,spell_paladin_hammerofwrath.jpg,136359,33,"[{'startTime': 4703930, 'endTime': 4716953}, {...",388256
7,Divine Resonance,0.23,384029,64,ability_bastion_paladin.jpg,90014,6,"[{'startTime': 4696986, 'endTime': 4711990}, {...",388256
8,Divine Steed,0.14,221886,2,ability_paladin_divinesteed.jpg,56055,7,"[{'startTime': 4689728, 'endTime': 4697737}, {...",388256
9,Light's Deliverance,0.14,433732,1,inv_glove_plate_raidpaladinmythic_q_01.jpg,53819,19,"[{'startTime': 4712296, 'endTime': 4715812}, {...",388256


In [298]:
list(map(get_buff_info_df, top_player_buff_info_lst))[0][['name', 'up_time_pct', 'type']].\
    to_dict(orient='records')

[{'name': 'Tempered Potion', 'up_time_pct': 0.15, 'type': 1},
 {'name': 'Winning Streak!', 'up_time_pct': 0.85, 'type': 1},
 {'name': 'Renew', 'up_time_pct': 0.13, 'type': 2},
 {'name': 'Divine Resonance', 'up_time_pct': 0.23, 'type': 64},
 {'name': "Sun's Avatar", 'up_time_pct': 0.0, 'type': 6},
 {'name': 'Divine Hammer', 'up_time_pct': 0.98, 'type': 2},
 {'name': 'Ascendance', 'up_time_pct': 0.21, 'type': 4},
 {'name': 'No, I Did That!', 'up_time_pct': 0.04, 'type': 1},
 {'name': 'Vampiric Speed', 'up_time_pct': 0.03, 'type': 1},
 {'name': "Stormbringer's Runed Citrine", 'up_time_pct': 0.08, 'type': 24},
 {'name': 'Divine Hymn', 'up_time_pct': 0.1, 'type': 2},
 {'name': 'Prayer of Mending', 'up_time_pct': 0.02, 'type': 2},
 {'name': 'Resonant Energy', 'up_time_pct': 0.27, 'type': 2},
 {'name': "Light's Deliverance", 'up_time_pct': 0.14, 'type': 1},
 {'name': 'Crystallization', 'up_time_pct': 1.0, 'type': 1},
 {'name': 'Shake the Heavens', 'up_time_pct': 0.98, 'type': 1},
 {'name': 'A

In [166]:
from warcraftlogs.client import WarcraftLogsClient
from warcraftlogs.query.player_analysis import analyze_player_performance

performance = analyze_player_performance(
    client=client,
    report_code=REPORT_CODE,
    fight_id=FIGHT_ID,
    source_id=SOURCE_ID
)

Analyzing Paladin - Retribution (ilvl: 658, bracket: 9)


KeyError: 'name'